In [28]:
import pandas as pd

In [29]:
data = pd.read_stata('../02. Datasets/ttest_with_result_mean_std.dta')
data.columns = data.iloc[0]
# Drop the first row
data = data.drop(0).reset_index(drop=True)

In [30]:
for i in range(1, 12):
    data.loc[i, 'SD'] = data.loc[i, 'Mean']
    data.loc[i, 'Mean'] = data.loc[i, 'Variable']
    data.loc[i, 'Variable'] = data.loc[i, 'abc']
    data.loc[i, 'abc'] = 0

    data.loc[12 + i, 'SD'] = data.loc[12 + i, 'Mean']
    data.loc[12 + i, 'Mean'] = data.loc[12 + i, 'Variable']
    data.loc[12 + i, 'Variable'] = data.loc[12 + i, 'abc']
    data.loc[12 + i, 'abc'] = 1

now we merge the table with the given label

In [31]:
df = pd.read_excel('../02. Datasets/variable__label_correspondence.xlsx')
df['varlab'] = df['varlab'].str.replace(r'^s\d+q\d+_\d+\s+', '', regex=True)
df['varlab'] = df['varlab'].str.replace(r's22q1_15a ', '') # s22q1_15a 
df['varlab'] = df['varlab'].str.replace(r'id11 ', '')

In [32]:
data['Variable'].replace('edchild pe~t', 'edchild_percent', inplace  = True)
data['Variable'].replace('accesscell~e', 'accesscellphone', inplace  = True)
data['Variable'].replace('eth hausa	', 'edchild_percent', inplace  = True)
data['Variable'].replace('accesscell~e', 'accesscellphone', inplace  = True)


In [33]:
data['Variable'] = data['Variable'] \
    .str.replace(' ', '_') \
    .str.replace('edchild_pe~t', 'edchild_percent') \
    .str.replace('eth_hausa', 'eth_hausa') \
    .str.replace('hhmem_no', 'hhmem_no') \
    .str.replace('accesscell~e', 'accesscellphone')


In [34]:
data = data.merge(df, left_on='Variable', right_on='name', how='left')

In [39]:
dataframe = data[['varlab', 'Mean', 'SD', 'abc']]

In [41]:
df_abc_0 = dataframe[dataframe['abc'] == 0].reset_index(drop=True)
df_abc_1 = dataframe[dataframe['abc'] == 1][['varlab', 'Mean', 'SD']].reset_index(drop=True)

df_abc_1.rename(columns={'Mean': 'Mean_abc', 'SD': 'SD_abc'}, inplace=True)

result_df = pd.merge(df_abc_0, df_abc_1, on='varlab', how='left')


In [43]:
result_df.drop(columns = ['abc'], inplace = True)

In [44]:
result_df

,varlab,Mean,SD,Mean_abc,SD_abc
0,Are you the household head?,0.560,0.497,0.547,0.498
1,Respondent is Hausa,0.715,0.452,0.721,0.449
2,Number of household members,8.422,4.054,8.328,4.074
3,Percentage of children under 15 who have some ...,0.279,0.276,0.269,0.270
4,Number of asset categories owned by household,4.990,1.609,4.979,1.575
5,Household experienced drought in past year,0.385,0.487,0.380,0.486
6,Household owns a cell phone (excluding group p...,0.296,0.457,0.295,0.457
7,Access to household or village-level cell phone,0.763,0.426,0.798,0.402
8,Respondent has used cell phone since last harvest,0.542,0.499,0.573,0.495
9,Respondent has made call,0.691,0.463,0.725,0.447


In [53]:
difference = pd.read_stata('../02. Datasets/Table1_PanelA_dta.dta')
difference.columns = difference.iloc[1]
# Drop the first row
difference = difference.drop(0).reset_index(drop=True)
difference = difference.drop(0).reset_index(drop=True)

In [54]:
difference

1,VARIABLES,age,hhhead,eth_hausa,hhmem_no,edchild_percent,assets,drought,cellphone,accesscellphone,usecellphone,makecall,receivecall
0,,,,,,,,,,,,,
1,abc,-0.36,-0.01,0.01,0.02,-0.00,-0.03,-0.03,-0.00,0.04*,0.04,0.03,0.03
2,,(0.93),(0.02),(0.03),(0.25),(0.02),(0.10),(0.03),(0.03),(0.02),(0.03),(0.04),(0.03)
3,_Iavcode_2,-0.66,-0.05,0.53***,0.71,-0.15***,0.78***,-0.26***,-0.07,-0.09,0.01,0.09,-0.19***
4,,(3.34),(0.06),(0.16),(0.72),(0.05),(0.23),(0.06),(0.11),(0.07),(0.11),(0.12),(0.07)
5,_Iavcode_3,-0.95,0.03,0.59***,1.04,-0.02,0.52**,-0.28***,-0.10,-0.03,-0.14,0.21**,-0.15
6,,(3.13),(0.07),(0.16),(0.74),(0.06),(0.22),(0.09),(0.12),(0.08),(0.11),(0.10),(0.10)
7,_Iavcode_4,3.82,0.12**,0.58***,1.82*,-0.09,0.40*,-0.26***,0.10,0.02,0.04,0.17,0.03
8,,(3.40),(0.05),(0.16),(0.99),(0.06),(0.21),(0.07),(0.11),(0.07),(0.11),(0.10),(0.05)
9,_Iavcode_5,2.70,0.01,-0.05,0.70,-0.11,0.03,-0.03,0.02,0.04,0.09,0.03,0.01
